In [6]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [7]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/usr/local/Caskroom/miniconda/base/envs/UCB/lib/python3.12/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

four_bedroom_avg = spark.sql("""
    SELECT date_format(sale_date, 'yyyy') AS year, round(avg(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year
    ORDER BY year
""")
four_bedroom_avg.show()

In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

three_bed_three_bath_avg = spark.sql("""
    SELECT date_built, round(avg(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
""")
three_bed_three_bath_avg.show()


In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

specific_home_avg = spark.sql("""
    SELECT date_built, round(avg(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
""")
specific_home_avg.show()


In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()
view_avg_price = spark.sql("""
    SELECT view, round(avg(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING avg_price >= 350000
    ORDER BY view DESC
""")
view_avg_price.show()
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")
print("Is table cached?", spark.catalog.isCached("home_sales"))



In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
view_avg_price_cached = spark.sql("""
    SELECT view, round(avg(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING avg_price >= 350000
    ORDER BY view DESC
""")
view_avg_price_cached.show()
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").parquet("home_sales_partitioned")


In [ ]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet("home_sales_partitioned")

In [ ]:
# 12. Create a temporary table for the parquet data.

parquet_df.createOrReplaceTempView("home_sales_parquet")


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
view_avg_price_parquet = spark.sql("""
    SELECT view, round(avg(price), 2) AS avg_price
    FROM home_sales_parquet
    GROUP BY view
    HAVING avg_price >= 350000
    ORDER BY view DESC
""")
view_avg_price_parquet.show()
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached

print("Is table uncached?", not spark.catalog.isCached("home_sales"))